A2 de Linguagens de Programação – 26/11/2018
--
Professor: Renato Rocha Souza  
--

Nome do Aluno(a):

In [1]:
meu_nome = input('Entre com o seu nome completo: ')
print('Olá, {}!'.format(meu_nome))
print('Bom trabalho!')

Entre com o seu nome completo: Tarla Lemos de Andrade
Olá, Tarla Lemos de Andrade!
Bom trabalho!


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import random
import re
import string
import time
import pymysql
import sqlite3
import getpass
import pandas.io.sql as psql
import networkx as nx
from IPython.display import clear_output
%matplotlib inline

Questão 1 (1,0 ponto):  

Utilizando Python + SQL, crie uma tabela "aluno" em banco de dados no sqlite que tenha os seguintes campos:

+ id 
+ nome
+ data_nascimento
+ matricula

Escolha os tipos de dados apropriados. O campo id é chave primária, e deve ser autoincrementável. 

Veja:  
https://sqlite.org/datatype3.html    
https://sqlite.org/autoinc.html  
https://www.tutorialspoint.com/sqlite/sqlite_update_query.htm  

In [2]:
conn = sqlite3.connect('../../Dados/meudatabase.sqlite')
cur = conn.cursor()

In [3]:
t = ''' CREATE TABLE IF NOT EXISTS aluno (
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY FIRST,
        nome VARCHAR(100) DEFAULT NULL,
        data_nasc DEFAULT NULL,
        matricula VARCHAR(50) DEFAULT NULL,
        )
        ENGINE=MyISAM DEFAULT CHARSET='utf8';'''

cur.execute(t)

OperationalError: near "AUTO_INCREMENT": syntax error

Questão 2 (1.5 ponto):  

Conecte ao banco SQLite "iris_database.sqlite"e responda, demonstrando os comandos Python + SQL apropriados:  

a) Há quantas "Species" diferentes na tabela "iris"?  
b) Quantos registros totais há na tabela "iris"?  

Considerando a superfície das pétalas e sépalas como retangular, responda:  

c) Qual espécie ("Species") contém, em média, a maior área de pétalas?  
d) Qual espécie ("Species") contém, em média, a maior área total (pétalas e sépalas)?  

(Por curiosidade, as espécies de Iris contém 3 pétalas e 3 sépalas)  

In [144]:
!pwd

/dados/home/tarlalemos/FGV_Programming_Languages/Python/Exercicios/Avaliacoes


In [4]:
conn = sqlite3.connect('../../Dados/iris_database.sqlite')
cur = conn.cursor()

In [7]:
cur.execute("describe iris")
for r in fetchall():
    print(r)

OperationalError: near "describe": syntax error

In [5]:
cur.execute('select distinct Species from  iris')
for r in cur.fetchall():
    print(r)

('Iris-setosa',)
('Iris-versicolor',)
('Iris-virginica',)


In [6]:
#b
cur.execute("select count(*) from iris")
for t in cur.fetchall():
    print(t)

(150,)


In [247]:
#c - Iris-virginica, Iris-virginica
media = cur.execute('select Species, avg(PetalLengthCm*PetalWidthCm) from iris group by Species')
for r in cur.fetchall():
    print(r)

('Iris-setosa', 0.36280000000000023)
('Iris-versicolor', 5.7204)
('Iris-virginica', 11.296199999999994)


In [249]:
cur.execute('select Species, avg(PetalLengthCm*PetalWidthCm)+avg(SepalLengthCm*SepalWidthCm) from iris group by Species')
for r in cur.fetchall():
    print(r)

('Iris-setosa', 17.571600000000007)
('Iris-versicolor', 22.2466)
('Iris-virginica', 30.980799999999995)


Questão 3 (1.5 pontos):  

Importe o arquivo ../../Dados/votesurvey.xls para um dataframe do Python usando o método apropriado, e responda às questões:  

a) Exiba as 15 primeiras linhas (essa é muito fácil!).  
b) Qual a média de salário desejado, dentre os eleitores de cada presidente?  
c) Qual fator tem maior influência sobre o salário atual? Sexo ou Idade?  
d) Crie uma nova coluna no dataframe original que exiba o salário atual dividido pela idade.  

In [314]:
vote = pd.read_excel('../../Dados/votesurvey.xls')

In [315]:
#a)tks
vote[:15]

,Gender,Age,Salary before Stern,Expected salary,Candidate
0,Male,27,60000,110000,Bush
1,Male,30,125000,125000,Bush
2,Male,27,50000,120000,Bush
3,Male,26,56000,100000,Bush
4,Male,27,82000,100000,Bush
5,Male,32,150000,150000,Bush
6,Female,24,55000,100000,Bush
7,Male,30,110000,150000,Bush
8,Male,26,48000,80000,Bush
9,Male,26,120000,80000,Bush


In [317]:
#b)
vote.groupby('Candidate').mean()['Expected salary']

Candidate
Bush                111666.666667
Gore                103055.555556
Refuse to answer    119500.000000
Undecided           103000.000000
Name: Expected salary, dtype: float64

In [325]:
#c) Pelo critério de soma dos erros ao quadrado é por gênero.
# Gênero:
media_por_genero = vote.groupby('Gender').mean()['Salary before Stern']
def erros_quadrados(vote, criterio):
    e = 0
    for x in range(len(vote)):
        e = e + (vote.loc[x, 'Expected salary']- criterio)**2
    return e
        
EQG = erros_quadrados(vote, media_por_genero)  

Gender
Female    1.401780e+11
Male      9.147509e+10
Name: Salary before Stern, dtype: float64


In [323]:
# idade:
media_por_idade = vote.groupby('Age').mean()['Salary before Stern']
erros_quadrados(vote,media_por_idade)

Age
24    1.617420e+11
25    1.750873e+11
26    8.756075e+10
27    1.466261e+11
28    1.079167e+11
29    7.235098e+10
30    3.912500e+10
32    1.111500e+11
33    1.471500e+11
Name: Salary before Stern, dtype: float64

In [320]:
media_por_genero

Gender
Female    61500.000000
Male      73705.882353
Name: Salary before Stern, dtype: float64

In [180]:
vote.loc[:, ['gender', 'Expected salary']]

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,gender,Expected salary
0,NaN,110000
1,NaN,125000
2,NaN,120000
3,NaN,100000
4,NaN,100000
5,NaN,150000
6,NaN,100000
7,NaN,150000
8,NaN,80000
9,NaN,80000


In [174]:
vote.groupby('Age').mean()['Expected salary']

Age
24     95000.000000
25    100000.000000
26    106250.000000
27    106666.666667
28    125833.333333
29    109285.714286
30    117500.000000
32    150000.000000
33    100000.000000
Name: Expected salary, dtype: float64

In [ ]:
sal = vote.pivot_table('births', index='Gender', columns='sex', aggfunc=sum)

In [251]:
#d)
vote['Salary/Age'] = [vote.loc[x,'Salary before Stern']/vote.loc[x,'Age'] for x in range(len(vote))] 

Questão 4 (1.5 pontos): 

Transforme o arquivo ../../Dados/DomCasmurroMachadoAssis.txt em uma string no Python e responda, usando, quando possível, expressões regulares:  

a) Quais as 10 palavras mais frequentes que _começam_ com "a" ou "A"?  
b) Quantas palavras distintas que começam com letras _maiúsculas_ há no texto?  
c) Quantas palavras há no texto com apenas uma; com apenas duas; e com apenas 3 vogais?  

In [47]:
f = open('../../Dados/DomCasmurroMachadoAssis.txt',encoding='utf-8')
texto = f.read()

In [287]:
#a)
# limpeza das palavras:
palavras = texto.split()
palavras = [p.strip(string.punctuation) for p in palavras]
filtro = [x for x in palavras if re.search('^[Aa].', x)]
sem_rep = list(set(filtro))
lis_dic = [(x,palavras.count(x)) for x in sem_rep]
lis_dic.sort(key=lambda x: x[1], )
ordenado = lis_dic[-1::-1]


In [282]:
ordenado[:10]

[('as', 497),
 ('ao', 328),
 ('ainda', 123),
 ('até', 97),
 ('agora', 94),
 ('aos', 85),
 ('assim', 81),
 ('and', 68),
 ('alguma', 67),
 ('aqui', 59)]

In [304]:
#b)
set_palavras = list(set(palavras))
T =[t for t in palavras if re.search(r'^[A-Z].*', t)]
len(T)

7075

In [309]:
#c)

# uma vogal:
uma_vogal =[t for t in palavras if re.search(r'^[^aeiou]*[aeiou][^aeiou]*$', t, re.IGNORECASE)]
duas_vogais = [t for t in palavras if re.search(r'^[^aeiou]*[aeiou][^aeiou]*[aeiou][^aeiou]*$', t, re.IGNORECASE)]
tres_vogais = [t for t in palavras if re.search(r'^[^aeiou]*[aeiou][^aeiou]*[aeiou][^aeiou]*[aeiou][^aeiou]*$', t, re.IGNORECASE)]

Qtd_uma_vogal = len(uma_vogal)
Qtd_duas_vogais = len(duas_vogais)
Qtd_tres_vogais = len(tres_vogais)

In [311]:
print(Qtd_uma_vogal) 
print(Qtd_duas_vogais)
print(Qtd_tres_vogais)

23037
22397
12716


Questão 5 (1.5 pontos):    

Utilizando apenas o Shell Linux, e utilizando o arquivo ../../Dados/DomCasmurroMachadoAssis.txt, diga os comandos para responder às seguintes perguntas:  

a) Quais as 10 palavras mais frequentes que _terminam_ com "a" ou "A"?  
b) Quantas palavras distintas que começam com _minúsculas_ há no texto?   
c) Como ordenar as palavras distintas por ordem de "rima"?  

In [99]:
#a)
!cat ../../Dados/DomCasmurroMachadoAssis.txt | tr -sc  '[a-záàâãéêíôóõúç][A-ZÁÀÃÂÉÊÍÓÔÚÇ]' '[\012*]' | grep '.[aA]$'| sort| uniq -c| sort -gr| sed 10q 

    623 da
    532 para
    468 era
    398 uma
    312 minha
    308 na
    177 ella
    170 casa
    133 outra
    130 ainda
sort: write failed: 'standard output': Broken pipe
sort: write error


In [104]:
#b)
!cat ../../Dados/DomCasmurroMachadoAssis.txt| tr -sc  '[a-záàâãéêíôóõúç][A-ZÁÀÃÂÉÊÍÓÔÚÇ]' '[\012*]'| grep '^[a-záàâãéêíôóõú].'| sort| uniq| wc -l

tr: write error: Broken pipe
tr: write error
cat: write error: Broken pipe
1795


In [108]:
#c)
!cat ../../Dados/DomCasmurroMachadoAssis.txt | tr ']' '\n'| tr -sc  '[a-záàâãéêíôóõúç][A-ZÁÀÃÂÉÊÍÓÔÚÇ]' '[\012*]'| sort| uniq -i| rev| sort| rev| sed 10q


rev: stdin: Invalid or incomplete multibyte or wide character

a
á
acaba
barba
ameaça
bocca
Bibliotheca
boneca
aconteça
uniq: write error: Broken pipe


Questão 6 (2.0 ponto):  

Conecte ao banco Python/Dados/chinook.sqlite e responda:

a) Leia para dataframes do Pandas as tabelas "invoices", "invoice_items", "customers" e "tracks"; e mostre quais as faixas compradas por clientes que têm o primeiro nome começando com a letra "M".  
b) Crie um histograma de faixas ("tracks") mais compradas.  
c) Escreva um script em Python + SQL para construir um dicionário tendo os nomes dos artistas ("artists") como chaves e uma lista dos álbuns ("albuns") respectivos como valores.

In [125]:
!pwd

/dados/home/tarlalemos/FGV_Programming_Languages/Python/Exercicios/Avaliacoes


In [329]:
conn = sqlite3.connect('../../Dados/chinook.sqlite')
cur = conn.cursor()

In [330]:
#a)
invoices = pd.read_sql_query('select * from invoices', conn)
invoice_items = pd.read_sql_query('select * from invoice_items', conn)
customers = pd.read_sql_query('select * from customers', conn)
tracks = pd.read_sql_query('select * from tracks', conn)

In [331]:
cur.execute('select customers.FirstName, tracks.name from customers,tracks join invoice_items.InvoiceId=invoices.InvoiceId where customers.FirstName like "M%" )

NameError: name 'invoices_items' is not defined

In [348]:
invoices.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [343]:
#b)

#Tracks mais compradas:
cur.execute('select InvoiceId,count(*) from invoice_items group by InvoiceId')
L=[]
for r in cur.fetchall():
    L.append(r)
L.sort(key=lambda x: x[1], )
L=L[-1::-1]
L = [L[x][0] for x in range(20)]
L #ID

#histograma:



[411,
 404,
 397,
 390,
 383,
 376,
 369,
 362,
 355,
 348,
 341,
 334,
 327,
 320,
 313,
 306,
 299,
 292,
 285,
 278]

In [ ]:
#c)

Auto-avaliação (1 ponto):    
Escreva sua auto-avaliação (mínimo de 3 linhas) e atribua a esta um valor entre 0 e 1, levando em consideração o aproveitamento na disciplina.

Precisaria de mais tempo para absorver o conteúdo da disciplina, considerando que o curso é muito denso e que não dediquei tempo suficiente para esta matéria. Consigo mapear todo o conteúdo dado, porém não o dominei (até o momento), e portanto gasto muito tempo fazendo as atividades.
pontuação: 0.65

### Escreva aqui